In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime
import chromedriver_autoinstaller
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

# 1. Opening Browser & scroll the job listing

In [2]:
# Define browser and action setup
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

# Define URL
linkedin_url = "https://www.linkedin.com/jobs/search/?keywords=data%20scientist&location=Indonesia"

# Action Steps
driver.maximize_window()
driver.get(linkedin_url) # Open web page

# Determine how many jobs we want to scrape, and calculate how many time we need to scroll down
no_of_jobs = 300
# int(driver.find_element_by_css_selector("h1>span").get_attribute("innerText"))
n_scroll = int(no_of_jobs/25)+1
print(n_scroll)
i = 1
driver.execute_script("return document.body.scrollHeight") #scroll to top
while i <= n_scroll:
    driver.execute_script("return document.body.scrollHeight")
    time.sleep(2) # wair for 2 seconds
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #scroll to the bottom of page
    time.sleep(3)
    i = i + 1
    try:
        button = driver.find_element(By.XPATH,"/html/body/div[1]/div/main/section[2]/button")
        #time.sleep(2)
        button.click()
        time.sleep(1)
        print("load more click")
    except:
        driver.execute_script("return document.body.scrollHeight")
        time.sleep(3)

print ("total jobs :")
jobs = driver.find_element(By.CLASS_NAME,"jobs-search__results-list").find_elements(By.TAG_NAME,'li') # return a list
print(len(jobs))

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_3104\912859545.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


13
load more click
load more click
load more click
load more click
load more click
load more click
load more click
load more click
total jobs :
374


# Get Main Attributes of each Job Listing 

In [3]:
job_id = []
job_title = []
company_name = []
location = []
date = []
job_link = []
for job in jobs:
    job_id0 = job.get_attribute('data-entity-urn')
    job_id.append(job_id0)

    job_title0 = job.find_element(By.CSS_SELECTOR,'h3').get_attribute('innerText')
    job_title.append(job_title0)

    company_name0 = job.find_element(By.CSS_SELECTOR,'h4').get_attribute('innerText')
    company_name.append(company_name0)

    location0 = job.find_element(By.CLASS_NAME,'job-search-card__location').get_attribute('innerText')
    location.append(location0)

    date0 = job.find_element(By.CSS_SELECTOR,'div>div>time').get_attribute('datetime')
    date.append(date0)

    job_link0 = job.find_element(By.CSS_SELECTOR,'a').get_attribute('href')
    job_link.append(job_link0)

In [4]:
jd = []
seniority = []
emp_type = []
job_func = []
industries = []
for item in range(len(jobs)):
    job_func0=[]
    industries0=[]
    # clicking job to view job details
    try:
        job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item+1}]/a/div[1]/img'
        job_click = job.find_element(By.XPATH,job_click_path).click()
    except:
        job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item+1}]/div/a'
    
    print(job_click_path)
    job_click = job.find_element(By.XPATH,job_click_path).click()
    time.sleep(3)
    try:
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[1]/div/div/section'
        jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
    except:
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[2]/div/div/section/div'
    
    jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
    is_benefit = True
    try:
        assert 'Base pay range' in jd0
    except:
        is_benefit = False
        
    if is_benefit==True :
        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[2]/div/div/section/div'
        jd0 = job.find_element(By.XPATH,jd_path).get_attribute('innerText')
        jd.append(jd0)
#         print(jd_path)
#         print("appended")
        jd_path2 = '/html/body/div[1]/div/section/div[2]/div/section[2]/div'
    else:
#         print(jd_path)
        jd.append(jd0)
#         print("appended")
        jd_path2 = '/html/body/div[1]/div/section/div[2]/div/section[1]/div'

    try:
        seniority_path = jd_path2 + '/ul/li[1]/span'
        seniority_class_name = 'description__job-criteria-text description__job-criteria-text--criteria'
        #seniority0 = job.find_element(By.XPATH,seniority_path).get_attribute('innerText')
        seniority0 = job.find_element(By.CLASS_NAME,seniority_class_name).get_attribute('innerText')
        seniority.append(seniority0)
    except:
        seniority.append('') #handling if seniority is not available
    
    try:
        emp_type_path = jd_path2 + '/ul/li[2]/span'
        emp_type_class_name = ''
        #emp_type0 = job.find_element(By.XPATH,emp_type_path).get_attribute('innerText')
        emp_type0 = job.find_element(By.CLASS_NAME,emp_type_class_name).get_attribute('innerText')
        emp_type.append(emp_type0)
    except:
        emp_type.append('') #handling if employment type is not available
    
    try:
        job_func_path = jd_path2 + '/ul/li[3]/span'
        job_func_class_name = ''
        #job_func0 = job.find_element(By.XPATH,job_func_path).get_attribute('innerText')
        job_func0 = job.find_element(By.CLASS_NAME,job_func_class_name).get_attribute('innerText')
        job_func.append(job_func0)
    except:
        job_func.append('') #handling if job function is not available
    
    try:
        industries_path = jd_path2 + '/ul/li[4]/span'
        industries_class_name = ''
        #industries0 = job.find_element(By.XPATH,industries_path).get_attribute('innerText')
        emp_type0 = job.find_element(By.CLASS_NAME,industries_class_name).get_attribute('innerText')
        industries.append(industries0)
    except:
        industries.append('')

/html/body/div[1]/div/main/section[2]/ul/li[1]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[2]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[3]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[4]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[5]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[6]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[7]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[8]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[9]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[10]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[11]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[12]/a/div[1]/img
/html/body/div[1]/div/main/section[2]/ul/li[13]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[14]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[15]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[16]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[17]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[18]/div/a
/html/body/div[1]/div/main/sec

/html/body/div[1]/div/main/section[2]/ul/li[151]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[152]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[153]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[154]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[155]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[156]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[157]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[158]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[159]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[160]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[161]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[162]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[163]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[164]/a/div[1]/img
/html/body/div[1]/div/main/section[2]/ul/li[165]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[166]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[167]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[168]/div/a
/ht

/html/body/div[1]/div/main/section[2]/ul/li[299]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[300]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[301]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[302]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[303]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[304]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[305]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[306]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[307]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[308]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[309]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[310]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[311]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[312]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[313]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[314]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[315]/div/a
/html/body/div[1]/div/main/section[2]/ul/li[316]/div/a
/html/body

In [5]:
job_data = pd.DataFrame({'ID': job_id,
                        'Date': date,
                        'Company': company_name,
                        'Title': job_title,
                        'Location': location,
                        'Description' : jd,
                        'Level': seniority,
                        'Type': emp_type,
                        'Function': job_func,
                        'Industry': industries
                        })

In [6]:
job_data.head(20)

,ID,Date,Company,Title,Location,Description,Level,Type,Function,Industry
0,None,2023-05-29,byOrange,Data Scientist,Jakarta,Who We Are:\n\n\n\n\nOrange provides your pers...,,,,
1,None,2023-06-05,ilmuOne Data,Junior Data Scientist (FT),Jakarta,Work Responsibilities\n\n\n\n\nAssist in metho...,,,,
2,None,2023-05-01,Indodana,Data Scientist,Jakarta,Job Description\n\n\n\n\nAnalyze data to bring...,,,,
3,None,2023-03-28,byOrange,Data Scientist (Digital Banking),Jakarta,"Responsibilities\n\n\n\n\nWork with large, com...",,,,
4,None,2023-05-01,Indodana,Data Scientist - Risk Platform,Jakarta,Job Description\n\n\n\n\nWe’re looking for fre...,,,,
5,None,2023-06-05,Frontiere,Junior Data Scientist,Roma,In Frontiere rendiamo accessibile l’innovazion...,,,,
6,None,2023-05-15,Devoteam G Cloud,Machine Learning Engineer,"Jakarta Raya, Indonesia",\nCompany Description\n\nDevoteam G Cloud is a...,,,,
7,None,2023-05-08,Artsy,Data Science & Analytics,Your,Artsy’s mission is to expand the art market to...,,,,
8,None,2023-06-05,MNC Financial Services (PT MNC Kapital Indones...,Python Engineer,Jakarta,"Responsibilities\n\n\n\n\nWrite effective, sca...",,,,
9,None,2023-05-22,Indodana,Senior Data Scientist,Jakarta,Company Description\n\n\n\n\nIndodana (PT Arth...,,,,


In [8]:
#change output file path to your desired path. Don't forget to put the filename and filetype also
job_data.to_excel('C:\\Users\\ThinkPad\\Documents\\DataScience.xlsx', index=False)